In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
from __future__ import print_function

from datetime import date
import datetime
#imports

# Ideas

Raise error when len does not match before and after merging datasets (means that a player wasn't found -- likely because he is a junior or has two last names, etc.)

# To Do

1. Address last name issue
2. Get pitcher xba onto
3. Scrape data instead of downloading CSVs

ON avoiding duplicate names --- maybe could change xba column in pitchers df before I merge, that way any batters with the name Luis Garcia would not have the column and be left out

In [2]:
batters_df = pd.read_csv('../../data/individual_batting_stats.csv')
pitchers_df = pd.read_csv('../../data/individual_pitching_stats.csv')
misc_batters = pd.read_csv('../../data/batters_teams.csv')
misc_pitchers = pd.read_csv('../../data/pitchers_teams.csv')

Pitcher Savant URL

https://baseballsavant.mlb.com/leaderboard/custom?year=2022&type=pitcher&filter=&sort=4&sortDir=asc&min=25&selections=p_total_pa,p_ab,p_total_hits,p_strikeout,p_walk,p_k_percent,p_bb_percent,batting_avg,slg_percent,on_base_percent,xba,xslg,woba,xwoba,xobp,xiso,wobacon,xwobacon,bacon,xbacon,xbadiff,xslgdiff,wobadif,exit_velocity_avg,launch_angle_avg,barrel_batted_rate,meatball_swing_percent,meatball_percent,&chart=false&x=xba&y=xba&r=no&chartType=beeswarm

Batter Savant URL 

https://baseballsavant.mlb.com/leaderboard/custom?year=2022&type=batter&filter=&sort=1&sortDir=asc&min=25&selections=b_ab,b_total_pa,b_total_hits,b_strikeout,b_walk,b_k_percent,b_bb_percent,batting_avg,slg_percent,on_base_percent,xba,xslg,woba,xwoba,xobp,xiso,wobacon,xwobacon,bacon,xbacon,xbadiff,xslgdiff,wobadif,exit_velocity_avg,launch_angle_avg,barrel_batted_rate,meatball_swing_percent,meatball_percent,&chart=false&x=xba&y=xba&r=no&chartType=beeswarm

In [3]:
y = batters_df[batters_df['last_name'].str.contains(" ")]
batters_df['last_name'] = [i.split(' ')[0] for i in batters_df['last_name']]
#The teams df does not have Jr., III for example listed on the names so I will cut them off before that so they match

In [5]:
len(batters_df)

445

# Probable Pitchers

In [6]:
d = datetime.datetime.now()
today = d.strftime("%Y-%m-%d")
#Get today's date

In [7]:
url = f'https://www.mlb.com/probable-pitchers/{today}'
#URL of where I will get the pitchers' names

In [8]:
res = requests.get(url)
soup = BeautifulSoup(res.content)
#Making request to the site and storing it in soup

In [9]:
# pitchers = soup.find_all('div', {'class': 'probable-pitchers__pitcher-name'})

In [10]:
pitchers = soup.find_all('div', {'class': 'probable-pitchers__pitcher-name'})
probable_pitchers = []
for i in pitchers:
    probable_pitchers.append(str(i))
#Finding all of the pitchers' names within that <a> tag
#Using for loop to convert the beautiful soup elements in the list to strings so I can get only the pitcher's name from there

In [11]:
probable_pitchers = [i.split('\n')[1] for i in probable_pitchers]
#Getting only the pitchers' names

In [12]:
for i in range(len(probable_pitchers)):
    if 'TBD' in probable_pitchers[i]:
        probable_pitchers[i] = '>TBD</a>'

In [13]:
probable_pitchers

['<a class="probable-pitchers__pitcher-name-link" href="/player/jon-gray-592351">Jon Gray</a>',
 '<a class="probable-pitchers__pitcher-name-link" href="/player/cal-quantrill-615698">Cal Quantrill</a>',
 '<a class="probable-pitchers__pitcher-name-link" href="/player/taylor-hearn-621368">Taylor Hearn</a>',
 '<a class="probable-pitchers__pitcher-name-link" href="/player/kirk-mccarty-668948">Kirk McCarty</a>',
 '<a class="probable-pitchers__pitcher-name-link" href="/player/tyler-gilbert-656457">Tyler Gilbert</a>',
 '<a class="probable-pitchers__pitcher-name-link" href="/player/graham-ashcraft-668933">Graham Ashcraft</a>',
 '>TBD</a>',
 '<a class="probable-pitchers__pitcher-name-link" href="/player/edward-cabrera-665795">Edward Cabrera</a>',
 '<a class="probable-pitchers__pitcher-name-link" href="/player/keegan-thompson-624522">Keegan Thompson</a>',
 '<a class="probable-pitchers__pitcher-name-link" href="/player/kyle-bradish-680694">Kyle Bradish</a>',
 '<a class="probable-pitchers__pitcher-

In [14]:
probable_pitchers = [i.split('</a')[0] for i in probable_pitchers] 
#Getting only the pitchers' names part 2

In [15]:
probable_pitchers = [i.split('>')[1] for i in probable_pitchers] 

In [16]:
probable_pitchers

['Jon Gray',
 'Cal Quantrill',
 'Taylor Hearn',
 'Kirk McCarty',
 'Tyler Gilbert',
 'Graham Ashcraft',
 'TBD',
 'Edward Cabrera',
 'Keegan Thompson',
 'Kyle Bradish',
 'Tarik Skubal',
 'Jose Quintana',
 'Dakota Hudson',
 'Jeffrey Springs',
 'Cole Irvin',
 'Kyle Wright',
 'Jameson Taillon',
 'Cole Sands',
 'Chris Flexen',
 'Justin Verlander',
 'Alek Manoah',
 'Brad Keller',
 'Mitch White',
 'Michael Kopech',
 'Ranger Suarez',
 'Jason Alexander',
 'Garrett Whitlock',
 'TBD',
 'Taijuan Walker',
 'Yu Darvish',
 'German Marquez',
 'Carlos Rodon']

In [17]:
probable_pitchers = [i.split() for i in probable_pitchers]
#Splitting name into first and last name columns

In [18]:
probable_pitchers

[['Jon', 'Gray'],
 ['Cal', 'Quantrill'],
 ['Taylor', 'Hearn'],
 ['Kirk', 'McCarty'],
 ['Tyler', 'Gilbert'],
 ['Graham', 'Ashcraft'],
 ['TBD'],
 ['Edward', 'Cabrera'],
 ['Keegan', 'Thompson'],
 ['Kyle', 'Bradish'],
 ['Tarik', 'Skubal'],
 ['Jose', 'Quintana'],
 ['Dakota', 'Hudson'],
 ['Jeffrey', 'Springs'],
 ['Cole', 'Irvin'],
 ['Kyle', 'Wright'],
 ['Jameson', 'Taillon'],
 ['Cole', 'Sands'],
 ['Chris', 'Flexen'],
 ['Justin', 'Verlander'],
 ['Alek', 'Manoah'],
 ['Brad', 'Keller'],
 ['Mitch', 'White'],
 ['Michael', 'Kopech'],
 ['Ranger', 'Suarez'],
 ['Jason', 'Alexander'],
 ['Garrett', 'Whitlock'],
 ['TBD'],
 ['Taijuan', 'Walker'],
 ['Yu', 'Darvish'],
 ['German', 'Marquez'],
 ['Carlos', 'Rodon']]

In [19]:
# probable_pitchers.insert(1, ['Roansy', 'Contreras'])
#Temporary

In [20]:
df_probable_pitchers = pd.DataFrame(probable_pitchers)
#Setting list as Pandas DF

In [21]:
df_probable_pitchers = df_probable_pitchers.rename(columns={0: "first_name", 1: "last_name"})
#Renaming column

# Get Pitchers' Teams

In [22]:
new_teams = soup.find_all('span', {'class':['probable-pitchers__team-name probable-pitchers__team-name--away', 'probable-pitchers__team-name probable-pitchers__team-name--home']})
teams_of_probable_pitchers = []
for i in new_teams:
    teams_of_probable_pitchers.append(str(i))
#Finding all of the pitchers' names within that <a> tag
#Using for loop to convert the beautiful soup elements in the list to strings so I can get only the pitcher's name from there

In [23]:
teams_of_probable_pitchers = [x.strip(' ') for x in teams_of_probable_pitchers]
#Getting only the pitchers' names

In [24]:
teams_of_probable_pitchers = [i.split('>\n')[1] for i in teams_of_probable_pitchers] 
#Getting only the pitchers' names

In [25]:
teams_of_probable_pitchers = teams_of_probable_pitchers = [x.strip('                             ') for x in teams_of_probable_pitchers]
#Stripping white space

In [26]:
teams_of_probable_pitchers = [i.split('\n')[0] for i in teams_of_probable_pitchers] 

In [27]:
len(teams_of_probable_pitchers)

32

In [28]:
df_probable_pitchers['team'] = teams_of_probable_pitchers
#Putting team on this way, now we just need to change it to the abbreviation

In [29]:
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Astros', 'HOU', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Nationals', 'WAS', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Giants', 'SF', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Blue Jays', 'TOR', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Angels', 'LAA', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'White Sox', 'CWS', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Guardians', 'CLE', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Athletics', 'OAK', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Twins', 'MIN', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Rays', 'TB', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Mariners', 'SEA', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Marlins', 'MIA', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Padres', 'SD', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Pirates', 'PIT', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Red Sox', 'BOS', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Orioles', 'BAL', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Braves', 'ATL', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Rangers', 'TEX', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Phillies', 'PHI', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Mets', 'NYM', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Yankees', 'NYY', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Royals', 'KC', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Cubs', 'CHC', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Brewers', 'MIL', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Reds', 'CIN', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Rockies', 'COL', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Tigers', 'DET', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Dodgers', 'LAD', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'D-backs', 'ARI', df_probable_pitchers['team'])
df_probable_pitchers['team'] = np.where(df_probable_pitchers['team'] == 'Cardinals', 'STL', df_probable_pitchers['team'])
#Changing team nickname to abbreviation so it will match the other DF -- Will clean this up in the future

In [30]:
misc_pitchers = misc_pitchers[['Player', 'Team']]
#All we need from this is their names and the team, because we will merge it with the other df to get the name on there

In [31]:
misc_pitchers[['first_name', 'last_name']] = misc_pitchers['Player'].str.split(' ', 1, expand=True)
#Separating 'Player' column into two columns for first and last to match probable_pitchers df

In [32]:
misc_pitchers = misc_pitchers.drop(columns='Player')
#We don't need this full name column anymore

In [33]:
misc_pitchers = misc_pitchers.rename(columns={"Team": "team"})
#Make this lowercase so it matches team column in probable pitchers

In [34]:
misc_pitchers = misc_pitchers[['last_name', 'first_name', 'team']]
#reordering columns

In [35]:
misc_pitchers.loc[misc_pitchers['last_name'] == 'Garcia']

,last_name,first_name,team
81,Garcia,Luis,HOU
205,Garcia,Yimi,TOR
346,Garcia,Rony,DET
409,Garcia,Jarlin,SF
437,Garcia,Luis,SD


# Get Batters' Teams

In [36]:
misc_batters = misc_batters[['Player', 'Team']]
#All we need from this is their names and the team, because we will merge it with the other df to get the name on there

In [37]:
misc_batters[['first_name', 'last_name']] = misc_batters['Player'].str.split(' ', 1, expand=True)
#Separating 'Player' column into two columns for first and last to match probable_pitchers df

In [38]:
misc_batters = misc_batters.drop(columns='Player')
#We don't need this full name column anymore

In [39]:
misc_batters = misc_batters[['last_name', 'first_name', 'Team']]
#Reordering columns

In [40]:
pitchers_df['first_name'] = pitchers_df['first_name'].str.strip()
batters_df['first_name'] = batters_df['first_name'].str.strip()
#Removing empty space in these column entries for example this makes ' Cabrera' => 'Cabrera'

batters_df and pitchers_df front names all have a space in front of them and that's why merge is not working

# Attaching according Team Name to each player in all DFs

The datasets on Baseball Savant do not include the team name with they players, so I will add them in

In [43]:
misc_pitchers = misc_pitchers.drop(misc_pitchers[misc_pitchers.last_name == 'Garcia'].index & misc_pitchers[misc_pitchers.first_name == 'Luis'].index & misc_pitchers[misc_pitchers.team == 'SD'].index)
#Dropping Padres' Luis Garcia to avoid issue of doubling when Astros' Luis Garcia pitches

C:\Users\User\AppData\Local\Temp/ipykernel_7272/877907739.py:1: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead
  misc_pitchers = misc_pitchers.drop(misc_pitchers[misc_pitchers.last_name == 'Garcia'].index & misc_pitchers[misc_pitchers.first_name == 'Luis'].index & misc_pitchers[misc_pitchers.team == 'SD'].index)


In [44]:
pitchers_df = pitchers_df.drop(pitchers_df[pitchers_df.last_name == 'Garcia'].index & pitchers_df[pitchers_df.first_name == 'Luis'].index & pitchers_df[pitchers_df.player_id == 472610 ].index)

C:\Users\User\AppData\Local\Temp/ipykernel_7272/1932908247.py:1: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead
  pitchers_df = pitchers_df.drop(pitchers_df[pitchers_df.last_name == 'Garcia'].index & pitchers_df[pitchers_df.first_name == 'Luis'].index & pitchers_df[pitchers_df.player_id == 472610 ].index)


In [45]:
df_probable_pitchers = pd.merge(df_probable_pitchers, misc_pitchers, on=['last_name', 'first_name', 'team'], how='left')
pitchers_df = pd.merge(pitchers_df, misc_pitchers, on=['last_name', 'first_name'], how='inner') #issue occurs on this merge
batters_df = pd.merge(batters_df, misc_batters, on=['last_name', 'first_name'], how='inner')
#Adding team to all of the players
#Joining probables as a left join so it will include all of the probables without exception -- may make one of their stats
# unavailable if they don't have enough PAs 

In [48]:
df_probable_pitchers = pd.merge(df_probable_pitchers, pitchers_df, on=['team', 'last_name', 'first_name'], how='left')
#Adding stats to probable pitchers

#This is what is adding the other Luis Garcia

# Which player to pick for BTS

In [49]:
pp_xba = df_probable_pitchers[['last_name', 'first_name', 'team', 'batting_avg', 'xba', 'xwoba', 'xslg', 'p_bb_percent']]
#How can I compare this to batters df??

One thing I could do is... switch the one column with the next, and then merge it with the batter's df
So for example if Oakland is playing Cleveland, I can switch the pitchers with each other. So Oakland's pitcher will be on Cleveland, and then I will have the pitcher xba to match with the team. Then I can basically set that for every member of the team, and then add that with the hitter xba and see if it meets the condition

In [51]:
pp_xba.sort_values(by='xba', ascending=False)

,last_name,first_name,team,batting_avg,xba,xwoba,xslg,p_bb_percent
2,Hearn,Taylor,TEX,0.295,0.295,0.396,0.537,11.0
4,Gilbert,Tyler,ARI,0.222,0.292,0.397,0.629,6.8
18,Flexen,Chris,SEA,0.275,0.291,0.379,0.538,7.5
11,Quintana,Jose,PIT,0.230,0.288,0.348,0.459,8.6
14,Irvin,Cole,OAK,0.256,0.288,0.366,0.533,5.8
12,Hudson,Dakota,STL,0.237,0.286,0.363,0.469,11.2
30,Marquez,German,COL,0.309,0.285,0.354,0.495,7.2
28,Walker,Taijuan,NYM,0.252,0.281,0.344,0.473,7.7
21,Keller,Brad,KC,0.248,0.273,0.327,0.441,7.1
24,Suarez,Ranger,PHI,0.277,0.272,0.345,0.450,10.2


In [67]:
batters_df.loc[batters_df['Team'] == 'TOR']

,last_name,first_name,player_id,year,b_ab,b_total_pa,b_total_hits,b_strikeout,b_walk,b_k_percent,...,wobadif,exit_velocity_avg,launch_angle_avg,barrel_batted_rate,meatball_swing_percent,meatball_percent,Unnamed: 32,Team,pitcher_xba,xba_total
68,Springer,George,543807,2022,174,200,48,43,16,21.5,...,0.021,89.9,16.8,9.6,81.8,7.2,NaN,TOR,0.273,0.540
161,Zimmer,Bradley,605548,2022,55,62,6,26,3,41.9,...,-0.011,87.4,3.5,6.7,69.2,5.5,NaN,TOR,0.273,0.405
164,Tapia,Raimel,606132,2022,148,158,36,32,7,20.3,...,-0.035,86.5,5.8,6.8,79.5,6.8,NaN,TOR,0.273,0.532
166,Hernandez,Teoscar,606192,2022,109,119,25,29,8,24.4,...,-0.056,91.1,7.6,13.8,80.6,7.3,NaN,TOR,0.273,0.545
212,Heineman,Tyler,623168,2022,42,48,8,6,4,12.5,...,-0.042,82.7,21.5,0.0,70.0,12.4,NaN,TOR,0.273,0.514
218,Biggio,Cavan,624415,2022,44,56,7,19,10,33.9,...,-0.039,89.9,18.5,8.0,72.2,7.2,NaN,TOR,0.273,0.463
241,Collins,Zack,641470,2022,62,69,12,28,6,40.6,...,0.010,93.5,26.4,11.8,77.8,9.2,NaN,TOR,0.273,0.465
276,Jansen,Danny,643376,2022,50,54,13,9,3,16.7,...,-0.029,89.8,23.4,24.4,65.0,9.6,NaN,TOR,0.273,0.566
295,Chapman,Matt,656305,2022,173,198,35,50,21,25.3,...,-0.062,92.0,19.1,10.5,80.0,7.5,NaN,TOR,0.273,0.516
359,Guerrero,Vladimir,665489,2022,187,215,47,38,23,17.7,...,-0.032,92.7,5.4,10.6,90.2,6.4,NaN,TOR,0.273,0.563


pp_xba['Team'] = " "
#New column to house when I switch the teams with each other
#I'm doing this so I can eventually merge it onto the batters' df with the hitters' team

# Getting Pitcher Stats onto Batter DF

FOR NOW, UNTIL I CLEAN THE BELOW UP A LITTLE MORE AND HAVE IT ITERATE THROUGH AND ADJUST TO THE NUMBER OF ROWS PER DAY, JUST USE THE AMOUNT OF ROWS THAT THE LEN() SHOWS AND COMMENT OUT THE REST

In [54]:
len(pp_xba)

32

In [55]:
pp_xba.head()

,last_name,first_name,team,batting_avg,xba,xwoba,xslg,p_bb_percent,Team
0,Gray,Jon,TEX,0.230,0.250,0.332,0.451,8.9,
1,Quantrill,Cal,CLE,0.236,0.268,0.340,0.451,8.7,
2,Hearn,Taylor,TEX,0.295,0.295,0.396,0.537,11.0,
3,McCarty,Kirk,CLE,NaN,NaN,NaN,NaN,NaN,
4,Gilbert,Tyler,ARI,0.222,0.292,0.397,0.629,6.8,


In [56]:
pp_xba.index = np.arange(1, len(pp_xba) + 1)

In [57]:
len(pp_xba)

32

In [58]:
pp_xba['Team'].iloc[1::2] = pp_xba['team'].iloc[::2]
pp_xba['Team'].iloc[::2] = pp_xba['team'].iloc[1::2]
#Putting odd teams on even teams and vice versa so that 'Team' reflects opposing pitcher

C:\Users\User\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [59]:
pp_xba = pp_xba.rename(columns={"batting_avg": "pitcher_batting_avg", "xba": "pitcher_xba"})
#Renaming columns for clarity

In [60]:
batters_df = pd.merge(batters_df, pp_xba[['Team','pitcher_xba']],on='Team', how='left')
#Getting pitcher data onto batters df

In [61]:
xba_sum = batters_df['xba'] + batters_df['pitcher_xba']
#Adding sum 

In [62]:
batters_df['xba_total'] = xba_sum
#Adding xba sum into the batters dataframe

# Which Hitters to pick for BTS

# Best Picks for Today

In [63]:
best_picks = batters_df[(batters_df['xba'] > .300) & (batters_df['pitcher_xba'] > .300) & (batters_df['xba_total'] > .650)]
best_picks

,last_name,first_name,player_id,year,b_ab,b_total_pa,b_total_hits,b_strikeout,b_walk,b_k_percent,...,wobadif,exit_velocity_avg,launch_angle_avg,barrel_batted_rate,meatball_swing_percent,meatball_percent,Unnamed: 32,Team,pitcher_xba,xba_total


In [64]:
second_best = batters_df[(batters_df['xba'] > .290) & (batters_df['pitcher_xba'] > .290) & (batters_df['xba_total'] > .600)]
second_best

,last_name,first_name,player_id,year,b_ab,b_total_pa,b_total_hits,b_strikeout,b_walk,b_k_percent,...,wobadif,exit_velocity_avg,launch_angle_avg,barrel_batted_rate,meatball_swing_percent,meatball_percent,Unnamed: 32,Team,pitcher_xba,xba_total
19,Brantley,Michael,488726,2022,173,199,48,21,23,10.6,...,-0.051,89.8,9.1,5.8,58.5,9.1,NaN,HOU,0.291,0.609
397,Alvarez,Yordan,670541,2022,169,197,48,33,26,16.8,...,-0.075,95.6,11.2,17.5,77.9,8.6,NaN,HOU,0.291,0.647


In [65]:
third_best = batters_df[(batters_df['xba'] > .275) & (batters_df['pitcher_xba'] > .275) & (batters_df['xba_total'] > .580)]
third_best.sort_values('xba_total', ascending = False)

,last_name,first_name,player_id,year,b_ab,b_total_pa,b_total_hits,b_strikeout,b_walk,b_k_percent,...,wobadif,exit_velocity_avg,launch_angle_avg,barrel_batted_rate,meatball_swing_percent,meatball_percent,Unnamed: 32,Team,pitcher_xba,xba_total
210,Reyes,Victor,622682,2022,24,26,7,4,2,15.4,...,-0.093,90.6,4.1,5.0,37.5,7.0,NaN,DET,0.288,0.697
397,Alvarez,Yordan,670541,2022,169,197,48,33,26,16.8,...,-0.075,95.6,11.2,17.5,77.9,8.6,NaN,HOU,0.291,0.647
214,Ramirez,Harold,623912,2022,127,136,36,16,6,11.8,...,-0.079,90.7,3.6,7.1,85.2,6.0,NaN,TB,0.286,0.640
162,Castro,Harold,605612,2022,102,107,30,19,3,17.8,...,-0.062,89.1,11.1,10.6,69.2,6.5,NaN,DET,0.288,0.624
118,Pederson,Joc,592626,2022,133,152,36,29,14,19.1,...,-0.062,94.6,11.9,20.8,82.6,7.8,NaN,SF,0.285,0.617
163,Arcia,Orlando,606115,2022,47,54,14,14,6,25.9,...,-0.054,93.8,11.0,11.8,85.0,8.9,NaN,ATL,0.288,0.615
19,Brantley,Michael,488726,2022,173,199,48,21,23,10.6,...,-0.051,89.8,9.1,5.8,58.5,9.1,NaN,HOU,0.291,0.609
208,Margot,Manuel,622534,2022,128,142,40,22,13,15.5,...,-0.004,90.9,14.0,4.7,69.8,7.7,NaN,TB,0.286,0.605
365,Gimenez,Andres,665926,2022,133,140,41,32,4,22.9,...,0.017,89.6,11.2,6.8,75.0,7.9,NaN,CLE,0.295,0.594
232,Meadows,Austin,640457,2022,101,116,27,14,12,12.1,...,-0.054,87.1,18.5,5.7,62.2,8.0,NaN,DET,0.288,0.594


In [66]:
next_best = batters_df[(batters_df['xba_total'] > .550)]
next_best.sort_values('xba_total', ascending = False)

,last_name,first_name,player_id,year,b_ab,b_total_pa,b_total_hits,b_strikeout,b_walk,b_k_percent,...,wobadif,exit_velocity_avg,launch_angle_avg,barrel_batted_rate,meatball_swing_percent,meatball_percent,Unnamed: 32,Team,pitcher_xba,xba_total
210,Reyes,Victor,622682,2022,24,26,7,4,2,15.4,...,-0.093,90.6,4.1,5.0,37.5,7.0,NaN,DET,0.288,0.697
397,Alvarez,Yordan,670541,2022,169,197,48,33,26,16.8,...,-0.075,95.6,11.2,17.5,77.9,8.6,NaN,HOU,0.291,0.647
214,Ramirez,Harold,623912,2022,127,136,36,16,6,11.8,...,-0.079,90.7,3.6,7.1,85.2,6.0,NaN,TB,0.286,0.640
162,Castro,Harold,605612,2022,102,107,30,19,3,17.8,...,-0.062,89.1,11.1,10.6,69.2,6.5,NaN,DET,0.288,0.624
118,Pederson,Joc,592626,2022,133,152,36,29,14,19.1,...,-0.062,94.6,11.9,20.8,82.6,7.8,NaN,SF,0.285,0.617
163,Arcia,Orlando,606115,2022,47,54,14,14,6,25.9,...,-0.054,93.8,11.0,11.8,85.0,8.9,NaN,ATL,0.288,0.615
19,Brantley,Michael,488726,2022,173,199,48,21,23,10.6,...,-0.051,89.8,9.1,5.8,58.5,9.1,NaN,HOU,0.291,0.609
208,Margot,Manuel,622534,2022,128,142,40,22,13,15.5,...,-0.004,90.9,14.0,4.7,69.8,7.7,NaN,TB,0.286,0.605
237,Anderson,Tim,641313,2022,163,173,58,20,6,11.6,...,0.001,88.2,4.2,9.1,74.4,8.0,NaN,CWS,0.270,0.602
365,Gimenez,Andres,665926,2022,133,140,41,32,4,22.9,...,0.017,89.6,11.2,6.8,75.0,7.9,NaN,CLE,0.295,0.594
